# template

In [211]:
import math
import pickle
import random
from typing import List, Tuple

import numpy as np
import torch
from catboost.datasets import msrank_10k
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from tqdm.auto import tqdm


class Solution:
    def __init__(self, n_estimators: int = 100, lr: float = 0.5, ndcg_top_k: int = 10,
                 subsample: float = 0.6, colsample_bytree: float = 0.9,
                 max_depth: int = 5, min_samples_leaf: int = 8):
        self._prepare_data()

        self.ndcg_top_k = ndcg_top_k
        self.n_estimators = n_estimators
        self.lr = lr
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf

        # допишите ваш код здесь

    def _get_data(self) -> List[np.ndarray]:
        train_df, test_df = msrank_10k()

        X_train = train_df.drop([0, 1], axis=1).values
        y_train = train_df[0].values
        query_ids_train = train_df[1].values.astype(int)

        X_test = test_df.drop([0, 1], axis=1).values
        y_test = test_df[0].values
        query_ids_test = test_df[1].values.astype(int)

        return [X_train, y_train, query_ids_train, X_test, y_test, query_ids_test]

    def _prepare_data(self) -> None:
        (X_train, y_train, self.query_ids_train,
            X_test, y_test, self.query_ids_test) = self._get_data()
        # допишите ваш код здесь
        pass

    def _scale_features_in_query_groups(self, inp_feat_array: np.ndarray,
                                        inp_query_ids: np.ndarray) -> np.ndarray:
        # допишите ваш код здесь 
        pass

    def _train_one_tree(self, cur_tree_idx: int,
                        train_preds: torch.FloatTensor
                        ) -> Tuple[DecisionTreeRegressor, np.ndarray]:
        # допишите ваш код здесь
        pass

    def _calc_data_ndcg(self, queries_list: np.ndarray,
                        true_labels: torch.FloatTensor, preds: torch.FloatTensor) -> float:
        # допишите ваш код здесь
        pass

    def fit(self):
        np.random.seed(0)
        # допишите ваш код здесь
        pass

    def predict(self, data: torch.FloatTensor) -> torch.FloatTensor:
        # допишите ваш код здесь
        pass

    def _compute_lambdas(self, y_true: torch.FloatTensor, y_pred: torch.FloatTensor) -> torch.FloatTensor:
        # допишите ваш код здесь
        pass

    def _ndcg_k(self, ys_true, ys_pred, ndcg_top_k) -> float:
        # допишите ваш код здесь
        pass

    def save_model(self, path: str):
        # допишите ваш код здесь
        pass

    def load_model(self, path: str):
        # допишите ваш код здесь
        pass


# code to send

In [89]:
import math
import pickle
import random
from typing import List, Tuple

import numpy as np
import torch
from catboost.datasets import msrank_10k
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from tqdm.auto import tqdm


class Solution:
    def __init__(self, n_estimators: int = 100, lr: float = 0.5, ndcg_top_k: int = 10,
                 subsample: float = 0.6, colsample_bytree: float = 0.9,
                 max_depth: int = 5, min_samples_leaf: int = 8):
        self._prepare_data()

        self.ndcg_top_k = ndcg_top_k
        self.n_estimators = n_estimators
        self.lr = lr
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf

        # допишите ваш код здесь
        self.subsample = subsample
        self.colsample_bytree = colsample_bytree
        
    def _get_data(self) -> List[np.ndarray]:
        train_df, test_df = msrank_10k()

        X_train = train_df.drop([0, 1], axis=1).values
        y_train = train_df[0].values
        query_ids_train = train_df[1].values.astype(int)

        X_test = test_df.drop([0, 1], axis=1).values
        y_test = test_df[0].values
        query_ids_test = test_df[1].values.astype(int)

        return [X_train, y_train, query_ids_train, X_test, y_test, query_ids_test]

    def _prepare_data(self) -> None:
        (X_train, y_train, self.query_ids_train,
            X_test, y_test, self.query_ids_test) = self._get_data()
        
        # scale train data
        self.X_train = torch.FloatTensor(self._scale_features_in_query_groups(
            inp_feat_array=X_train, inp_query_ids=self.query_ids_train))
#         self.ys_train = torch.FloatTensor(y_train).t()
        self.ys_train = torch.FloatTensor(y_train).reshape(-1,1)
        
        # scale test data
        self.X_test = torch.FloatTensor(self._scale_features_in_query_groups(
            inp_feat_array=X_test, inp_query_ids=self.query_ids_test))
#         self.ys_test = torch.FloatTensor(y_test).t()
        self.ys_test = torch.FloatTensor(y_test).reshape(-1,1)
        pass

    def _scale_features_in_query_groups(self, inp_feat_array: np.ndarray,
                                        inp_query_ids: np.ndarray) -> np.ndarray:
        # get unique ids
        inp_query_ids_uniq = np.unique(inp_query_ids)
        
        # scale each group by id
        for id_i in inp_query_ids_uniq:
            scaler = StandardScaler()
            inp_feat_array[inp_query_ids == id_i, :] = \
                scaler.fit_transform(inp_feat_array[inp_query_ids == id_i, :])

        return inp_feat_array

    
    
    def _train_one_tree(self, cur_tree_idx: int,
                        train_preds: torch.FloatTensor
                        ) -> Tuple[DecisionTreeRegressor, np.ndarray]:
        # допишите ваш код здесь
        random.seed(cur_tree_idx)
        
        ## compute lambdas with train_preds
        ids_uniq = np.unique(self.query_ids_train)
        
#         print('self.train_preds')
#         print(self.train_preds)
#         print('np.shape(self.train_preds)')
#         print(np.shape(self.train_preds))
        if all(np.array(self.train_preds) == 0):
            lambdas_upd = self.ys_train
        else:
            lambdas_list = []

            for n_id in ids_uniq:

                X_train_id = self.X_train[self.query_ids_train == n_id]
                ys_train_id = self.ys_train[self.query_ids_train == n_id]
                train_preds_id = train_preds[self.query_ids_train == n_id]

                lambdas_list.append(\
                    self._compute_lambdas(y_true=ys_train_id, y_pred=train_preds_id))

#             lambdas_upd = np.concatenate(lambdas_list)
            lambdas_upd = np.concatenate(lambdas_list) * -1
#             lambdas_upd = np.concatenate(lambdas_list) * -1 * self.lr
        
        ## set train data
        N, M = np.shape(self.X_train)
        
        random.seed(cur_tree_idx)
        sample_ids = random.sample(range(N), int(N * self.subsample))
        random.seed(cur_tree_idx)
        colsample_ids = np.array(random.sample(range(M), int(M * self.colsample_bytree)))
        
#         print('np.shape(self.X_train)')
#         print(np.shape(self.X_train))
#         print('np.shape(sample_ids)')
#         print(np.shape(sample_ids))
#         print('np.shape(colsample_ids)')
#         print(np.shape(colsample_ids))
        X_train = self.X_train[sample_ids, :][:, colsample_ids]
#         ys_train = self.ys_train[sample_ids, ]
#         ys_train = train_preds[sample_ids]
        ys_train = lambdas_upd[sample_ids]
        
#         print('ys_train_train_one_tree')
#         print(ys_train)
        
        # fit one tree & predict
        single_tree = DecisionTreeRegressor(max_depth=self.max_depth,\
                                            min_samples_leaf=self.min_samples_leaf,\
                                            random_state=cur_tree_idx)
        single_tree.fit(X=X_train, y=ys_train)
        
        # return tree and features ids
        return (single_tree, colsample_ids)

    
#     def _compute_lambdas(self, y_true: torch.FloatTensor, y_pred: torch.FloatTensor) -> torch.FloatTensor:
#         # допишите ваш код здесь
#         pass
    
    def _compute_lambdas(self, y_true, y_pred, ndcg_scheme='exp2'):
        # рассчитаем нормировку, IdealDCG
#         print('y_true')
#         print(y_true)
#         print('y_pred')
#         print(y_pred)
        if all(np.array(y_true) == 0):
            ideal_dcg = 1
        else:
            ideal_dcg = self._compute_ideal_dcg(y_true.reshape(1,-1)[0],\
                                            ndcg_scheme=ndcg_scheme)
        
#         print('ideal_dcg')
#         print(ideal_dcg)
        N = 1 / ideal_dcg

        # рассчитаем порядок документов согласно оценкам релевантности
        _, rank_order = torch.sort(y_true, descending=True, axis=0)
        rank_order += 1

        with torch.no_grad():
            # получаем все попарные разницы скоров в батче
#             pos_pairs_score_diff = 1.0 + torch.exp((y_pred - y_pred.t()))
            pos_pairs_score_diff = 1.0 + torch.exp((y_pred - y_pred.reshape(-1,1)))
#             print('pos_pairs_score_diff')
#             print(pos_pairs_score_diff)
            
            # поставим разметку для пар, 1 если первый документ релевантнее
            # -1 если второй документ релевантнее
            Sij = self._compute_labels_in_batch(y_true)
#             print('Sij')
#             print(Sij)
            # посчитаем изменение gain из-за перестановок
            gain_diff = self._compute_gain_diff(y_true, ndcg_scheme)
#             print('gain_diff')
#             print(gain_diff)
            
            # посчитаем изменение знаменателей-дискаунтеров
#             decay_diff = (1.0 / torch.log2(rank_order + 1.0)) - (1.0 / torch.log2(rank_order.t() + 1.0))
            decay_diff = (1.0 / torch.log2(rank_order + 1.0)) - \
                (1.0 / torch.log2(rank_order.reshape(1,-1) + 1.0))
#             print('decay_diff')
#             print(decay_diff)
            # посчитаем непосредственное изменение nDCG
            delta_ndcg = torch.abs(N * gain_diff * decay_diff)
            # посчитаем лямбды
            lambda_update =  (0.5 * (1 - Sij) - 1 / pos_pairs_score_diff) * delta_ndcg
#             print('lambda_update')
#             print(lambda_update)
            lambda_update = torch.sum(lambda_update, dim=1, keepdim=True)

#             return Sij, gain_diff, decay_diff, delta_ndcg, lambda_update
        return lambda_update
    
    def _compute_labels_in_batch(self, y_true):

        # разница релевантностей каждого с каждым объектом
#         print('y_true_in_compute_labels_in_batch')
#         print(y_true)
#         rel_diff = y_true - y_true.t()
        rel_diff = y_true - y_true.reshape(1,-1)
#         print('rel_diff')
#         print(rel_diff)
        
        # 1 в этой матрице - объект более релевантен
        pos_pairs = (rel_diff > 0).type(torch.float32)

        # 1 тут - объект менее релевантен
        neg_pairs = (rel_diff < 0).type(torch.float32)
        Sij = pos_pairs - neg_pairs
        return Sij

    def _compute_gain_diff(self, y_true, gain_scheme):
        if gain_scheme == "exp2":
#             gain_diff = torch.pow(2.0, y_true) - torch.pow(2.0, y_true.t())
            gain_diff = torch.pow(2.0, y_true) - torch.pow(2.0, y_true.reshape(1,-1))
        elif gain_scheme == "diff":
            gain_diff = y_true - y_true.t()
        else:
            raise ValueError(f"{gain_scheme} method not supported")
        return gain_diff
    
    def _calc_data_ndcg(self, queries_list: np.ndarray,
                        true_labels: torch.FloatTensor, preds: torch.FloatTensor) -> float:
        
        ndcgs = []
        
        # допишите ваш код здесь
        with torch.no_grad():

#             ids_test_uniq = np.unique(self.query_ids_test)

#             for id_test in ids_test_uniq:
#                 cur_X_test = self.X_test[self.query_ids_test == id_test, :]
#                 valid_pred = self.predict(cur_X_test)
#                 cur_ndcg = self._ndcg_k(\
#                     ys_true=self.ys_test.reshape(1,-1)[0][self.query_ids_test == id_test],\
#                     ys_pred=valid_pred,\
#                     ndcg_top_k=10)
                
            ids_test_uniq = np.unique(queries_list)

            for id_test in ids_test_uniq:
                cur_ndcg = self._ndcg_k(\
                    ys_true=true_labels[queries_list == id_test],\
                    ys_pred=preds[queries_list == id_test],\
                    ndcg_top_k=self.ndcg_top_k)
                
                ndcgs.append(cur_ndcg)
                
        return np.mean(ndcgs)

    def fit(self):
        np.random.seed(0)
        # допишите ваш код здесь
        self.train_preds = torch.zeros(np.shape(self.ys_train)[0])
        self.features_ids = []
        self.trees = []
        self.best_ndcg = 0
        self.best_n_trees = 1
        self.ndcg_list = []
        
        
        for i in range(self.n_estimators):
            single_tree, features_ids = self._train_one_tree(\
                cur_tree_idx=i, train_preds=self.train_preds)
            self.trees.append(single_tree)
            self.features_ids.append(features_ids)
            
            ## calculate new tree prediction & update train_preds
            new_ys_preds = single_tree.predict(self.X_train[:, features_ids])
#             self.train_preds += new_ys_preds
#             self.train_preds += new_ys_preds * self.lr
            
#             print('new_ys_preds')
#             print(new_ys_preds)
#             print('self.train_preds')
#             print(self.train_preds)
            
            ## calc new ndcg and update best & best_n_tree with _calc_data_ndcg
            
            # apply _calc_data_ndcg
            ndcg_cur_train = self._calc_data_ndcg(\
                queries_list=self.query_ids_train, true_labels=self.ys_train,\
                preds=self.predict(self.X_train))
            
            ndcg_cur_test = self._calc_data_ndcg(\
                queries_list=self.query_ids_test, true_labels=self.ys_test,\
                preds=self.predict(self.X_test))
            
            ndcg_cur = ndcg_cur_test
            self.ndcg_list.append(ndcg_cur)
            self.best_ndcg = max(self.ndcg_list)
            self.best_n_trees = np.argmax(self.ndcg_list) + 1
            
# #             print('i_in_fit_fun')
#             print(i)
# #             print('ndcg_cur_in_fit_fun')
#             print(ndcg_cur)
            print("n_tree: {}, train_ndcg_k: {}, test_ndcg_k: {}".format(\
                i, ndcg_cur_train, ndcg_cur_test))
        
        self.trees = self.trees[:self.best_n_trees]
        self.features_ids = self.features_ids[:self.best_n_trees]
        self.n_estimators = self.best_n_trees
        
#         self.trees = trees_list

    def predict(self, data: torch.FloatTensor) -> torch.FloatTensor:
        # допишите ваш код здесь
        preds = torch.zeros(np.shape(data)[0])
#         for i in range(self.n_estimators):
        for i in range(len(self.trees)):
#             print('len(self.trees)')
#             print(len(self.trees))
            ## calculate new tree prediction & update train_preds
#             print("n_estimator_i")
#             print(i)
            new_ys_preds = self.trees[i].predict(data[:, self.features_ids[i]])
#             self.train_preds += new_ys_preds * self.lr
            preds += new_ys_preds * self.lr
            
        return preds
            
#     def _ndcg_k(self, ys_true, ys_pred, ndcg_top_k) -> float:
#         # допишите ваш код здесь
#         pass
    
    def _ndcg_k(self, ys_true: torch.Tensor, ys_pred: torch.Tensor,
                ndcg_top_k: int) -> float:
        # допишите ваш код здесь
        ys_true = torch.reshape(ys_true, (-1,))
        ys_pred = torch.reshape(ys_pred, (-1,))

        ys_pred_sorted = torch.sort(ys_pred, descending=True)
        ys_true_sorted = ys_true[ys_pred_sorted[1]]
        ys_true_sorted_sep = torch.sort(ys_true, descending=True)[0]

        dcg_act = self._dcg(ys_true=ys_true_sorted[:self.ndcg_top_k],\
                            ys_pred=ys_pred_sorted[0][:self.ndcg_top_k],\
                            gain_scheme='exp2')

        dcg_max = self._dcg(ys_true=ys_true_sorted_sep[:self.ndcg_top_k],\
                            ys_pred=ys_true_sorted_sep[:self.ndcg_top_k],\
                            gain_scheme='exp2')

#         try:
#             ndcg_k = dcg_act / dcg_max
#         except:
#             ndcg_k = 0
#         if np.isnan(ndcg_k):
#             ndcg_k = 0
        
#         if (dcg_act == 0):
#             ndcg_k = 0
        if (dcg_max == 0):
            ndcg_k = 1
        else:
            ndcg_k = dcg_act / dcg_max
        
        return ndcg_k

    def _ndcg(self, ys_true: torch.Tensor, ys_pred: torch.Tensor, gain_scheme: str = 'const') -> float:
#         try:
        dcg_val = self._dcg(ys_true=ys_true, ys_pred=ys_pred, gain_scheme=gain_scheme)
        ideal_dcg_val = self._dcg(ys_true=ys_true, ys_pred=ys_true, gain_scheme=gain_scheme)
        
#         print('dcg_val')
#         print(dcg_val)
#         print('ideal_dcg_val')
#         print(ideal_dcg_val)
        
#         if (dcg_val == 0):
#             return dcg_val
        if (ideal_dcg_val == 0):
            return 1
        
        else:
            return dcg_val / ideal_dcg_val
    
    def _compute_gain(self, y_value: float, gain_scheme: str) -> float:
#         return 2 ** y_value - 1.
        if gain_scheme == 'exp2':
            return 2 ** y_value - 1.
        else:
            return y_value + 0.

    def _dcg(self, ys_true: torch.Tensor, ys_pred: torch.Tensor, gain_scheme: str) -> float:
        ys_pred_sorted = torch.sort(ys_pred, descending=True)
#         print('ys_pred_sorted')
#         print(ys_pred_sorted)
        
        log2_list = [math.log2(x) for x in range(2, len(ys_pred) + 2)]

        dcg_val = 0.
        for i in range(len(log2_list)):
            dcg_val += self._compute_gain(ys_true[ys_pred_sorted[1]][i].item(), \
                                          gain_scheme=gain_scheme) / log2_list[i]
            
#         print('dcg_val')
#         print(dcg_val)
        
#         if (dcg_val == 0):
# #             dcg_val = -1
#             dcg_val = 0.00001
        return dcg_val
    
    def _compute_ideal_dcg(self, y_true, ndcg_scheme='exp2'):
#         print(y_true)
#         try:
        return self._dcg(ys_true=y_true, ys_pred=y_true, gain_scheme=ndcg_scheme)
#         except:
#             return 1
    
    
    def save_model(self, path: str):
        # допишите ваш код здесь
        ## save model
        state = {
            'trees': self.trees, 
            'features_ids': self.features_ids,
            'ndcg_top_k': self.ndcg_top_k,
#             'best_n_trees': self.best_n_trees,
            'n_estimators': self.n_estimators,
            'lr': self.lr}
        
        f = open(path, 'wb')
        pickle.dump(state, f)
        
        pass

    def load_model(self, path: str):
        # допишите ваш код здесь
        f = open(path, 'rb')
        state = pickle.load(f)
        
        self.trees = state['trees']
        self.features_ids = state['features_ids']
        self.ndcg_top_k = state['ndcg_top_k']
#         self.best_n_trees = state['best_n_trees']
        self.lr = state['lr']
        
#         return state
        pass
    
#     def save_obj(obj, name):
#         with open(name + '.pkl', 'wb') as f:
#             pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

#     def load_obj(name):
#         with open(name + '.pkl', 'rb') as f:
#             return pickle.load(f)

In [97]:
tst_sol1 = Solution(n_estimators=100)
# tst_sol1 = Solution(n_estimators=7)

In [98]:
%%time
tst_sol1.fit()

n_tree: 0, train_ndcg_k: 0.45968059872384603, test_ndcg_k: 0.3295782293708683
n_tree: 1, train_ndcg_k: 0.5216392964861525, test_ndcg_k: 0.336569784676915
n_tree: 2, train_ndcg_k: 0.5221855200242471, test_ndcg_k: 0.36380267274701583
n_tree: 3, train_ndcg_k: 0.5368954236716924, test_ndcg_k: 0.37187789290166084
n_tree: 4, train_ndcg_k: 0.5403321922419148, test_ndcg_k: 0.3704574068159138
n_tree: 5, train_ndcg_k: 0.5453128913862544, test_ndcg_k: 0.3725899192169184
n_tree: 6, train_ndcg_k: 0.5485454855826337, test_ndcg_k: 0.3870515435122703
n_tree: 7, train_ndcg_k: 0.5580627654957582, test_ndcg_k: 0.3934614396866454
n_tree: 8, train_ndcg_k: 0.5559738142964052, test_ndcg_k: 0.40327906989800755
n_tree: 9, train_ndcg_k: 0.5661915736895955, test_ndcg_k: 0.4010444691582316
n_tree: 10, train_ndcg_k: 0.5690039433770224, test_ndcg_k: 0.4023557874145436
n_tree: 11, train_ndcg_k: 0.5679965689200871, test_ndcg_k: 0.40419252489533336
n_tree: 12, train_ndcg_k: 0.5631563454983873, test_ndcg_k: 0.406189332

In [99]:
tst_sol1.best_n_trees, tst_sol1.best_ndcg

(20, 0.43173542982162266)

In [96]:
# %%time
# tst_sol1.fit()

n_tree: 0, train_ndcg_k: 0.45968059872384603, test_ndcg_k: 0.3295782293708683
n_tree: 1, train_ndcg_k: 0.5216392964861525, test_ndcg_k: 0.336569784676915
n_tree: 2, train_ndcg_k: 0.5221855200242471, test_ndcg_k: 0.36380267274701583
n_tree: 3, train_ndcg_k: 0.5368954236716924, test_ndcg_k: 0.37187789290166084
n_tree: 4, train_ndcg_k: 0.5403321922419148, test_ndcg_k: 0.3704574068159138
n_tree: 5, train_ndcg_k: 0.5453128913862544, test_ndcg_k: 0.3725899192169184
n_tree: 6, train_ndcg_k: 0.5485454855826337, test_ndcg_k: 0.3870515435122703
CPU times: user 6.8 s, sys: 45.3 ms, total: 6.84 s
Wall time: 3.61 s


In [38]:
# tst_sol1.save_model(path='lambdamart_01.pkl')

In [50]:
tst_sol1.load_model(path='lambdamart_01.pkl')

In [51]:
# tst_sol1.save_model(path='lambdamart_02.pkl')

(69, 0.4321047855828739)

# funs tests (draft)

In [25]:
N = int(np.shape(tst_sol1.X_train)[0] * 0.9)
N = np.shape(tst_sol1.X_train)[0]
N

10000

In [28]:
# np.random()
random.randint(0, 10)
smpl1 = random.sample(range(1000), 10)
smpl1

[778, 127, 18, 845, 27, 423, 744, 182, 766, 725]

In [29]:
tst_sol1.X_train[smpl1, ]

tensor([[ 0.3524, -0.4220,  0.4964,  ..., -0.1267, -0.0789, -0.4441],
        [ 0.6456,  0.4900,  0.6141,  ...,  0.0000, -0.0974,  1.0093],
        [ 0.3161, -0.2350, -0.3139,  ..., -0.1118, -0.1959, -0.2662],
        ...,
        [ 0.6456,  1.5499,  2.2621,  ...,  0.0000, -0.0977, -0.4654],
        [ 0.3524, -0.4220,  0.4964,  ..., -0.1267, -0.0789, -0.4441],
        [ 0.3524, -0.4220,  0.4964,  ..., -0.1267, -0.0789, -0.4441]])

In [35]:
n, m = np.shape(tst_sol1.X_train[smpl1, ])
np.array(random.sample(range(m), 10))

array([ 3, 55, 52, 57, 46, 97, 73, 39, 77,  6])

In [39]:
np.ndarray([1,2,3], dtype=int)

array([[[140260008804688,               0,               0],
        [              0,               0,               0]]])

In [42]:
np.ndarray([1,2,3], dtype=int)

TypeError: data type not understood

In [46]:
('a', [1, 2])

('a', [1, 2])

In [102]:
all(tst_sol1.train_preds == 0)

True

In [171]:
random.sample(range(2), 2)

[1, 0]

In [176]:
np.shape(tst_sol1.X_train[random.sample(range(5000), 1000), ])

torch.Size([1000, 136])

In [15]:
tst_sol1.X_train[random.sample(range(5000), 1000), random.sample(range(10), 2)]
# random.sample(range(100), 80)

IndexError: shape mismatch: indexing tensors could not be broadcast together with shapes [1000], [2]

In [12]:
np.shape(tst_sol1.X_train[random.sample(range(5000), 1000), random.sample(range(10), 2)[:, np.newaxis]])

TypeError: list indices must be integers or slices, not tuple

In [16]:
x = torch.randn(4, 5)
x
# torch.index_select(x, 1, (1,3))

tensor([[-0.2744,  1.2814,  1.2673,  0.4213,  0.7169],
        [ 1.2463, -1.1928,  1.2915,  1.2466,  0.1801],
        [ 1.9949, -0.0971,  0.2938, -2.0156,  1.6405],
        [ 0.0980, -2.7853, -0.0406, -0.0128,  0.6291]])

In [22]:
x[:, [1, 3]]

tensor([[ 1.2814,  0.4213],
        [-1.1928,  1.2466],
        [-0.0971, -2.0156],
        [-2.7853, -0.0128]])

In [24]:
x[[0, 2], [1, 3]]

tensor([ 1.2814, -2.0156])

In [27]:
x[:, [1, 3]][[0, 2], ]

tensor([[ 1.2814,  0.4213],
        [-0.0971, -2.0156]])

In [29]:
x[:, [1, 3]][[0, 2], :]

tensor([[ 1.2814,  0.4213],
        [-0.0971, -2.0156]])

In [30]:
x[[0, 2], :][:, [1, 3]]

tensor([[ 1.2814,  0.4213],
        [-0.0971, -2.0156]])

In [39]:
z = []
z.append(np.array([1,2,3]))
z.append(np.array([1,2,3]))
z

[array([1, 2, 3]), array([1, 2, 3])]

In [87]:
tst_t = torch.tensor([2., 2., 0., 2., 1., 1., 1., 2., 1., 0., 0., 0., 0., 0., 0., 0., 0., 2.,\
        0., 0., 2., 2., 0., 1., 0., 1., 2., 0., 0., 0., 1., 1., 0., 1., 0., 0.,\
        0., 0., 0., 0., 0., 1., 0., 0., 0., 2., 3., 0., 0., 0., 0., 0., 1., 0.,\
        0., 0., 2., 1., 2., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0.,\
        0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 1., 1., 0.])
tst_t

tensor([2., 2., 0., 2., 1., 1., 1., 2., 1., 0., 0., 0., 0., 0., 0., 0., 0., 2.,
        0., 0., 2., 2., 0., 1., 0., 1., 2., 0., 0., 0., 1., 1., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 2., 3., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 2., 1., 2., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 1., 1., 0.])

In [91]:
tst_t

tensor([2., 2., 0., 2., 1., 1., 1., 2., 1., 0., 0., 0., 0., 0., 0., 0., 0., 2.,
        0., 0., 2., 2., 0., 1., 0., 1., 2., 0., 0., 0., 1., 1., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 2., 3., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 2., 1., 2., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 1., 1., 0.])

In [97]:
# tst_t2 = torch.LongTensor([[5, 3, 2, 5, 1, 1]]).reshape(-1, 1)
tst_t2 = torch.Tensor([[5, 3, 2, 5, 1, 1]]).reshape(-1, 1)
tst_t2

tensor([[5.],
        [3.],
        [2.],
        [5.],
        [1.],
        [1.]])

In [99]:
tst_t3 = torch.Tensor([5, 3, 2, 5, 1, 1])
tst_t3

tensor([5., 3., 2., 5., 1., 1.])

In [188]:
tst_t3 - tst_t3.reshape(-1, 1)

tensor([[ 0., -2., -3.,  0., -4., -4.],
        [ 2.,  0., -1.,  2., -2., -2.],
        [ 3.,  1.,  0.,  3., -1., -1.],
        [ 0., -2., -3.,  0., -4., -4.],
        [ 4.,  2.,  1.,  4.,  0.,  0.],
        [ 4.,  2.,  1.,  4.,  0.,  0.]])

In [189]:
tst_t3 - tst_t3.reshape(1, -1)

tensor([[0., 0., 0., 0., 0., 0.]])

In [129]:
torch.zeros(np.shape(tst_sol1.ys_train.reshape(1,-1)))
torch.zeros(np.shape(tst_sol1.ys_train)[0])

tensor([0., 0., 0.,  ..., 0., 0., 0.])

In [183]:
np.shape(torch.zeros(np.shape(tst_sol1.ys_train.reshape(1,-1))))

torch.Size([1, 10000])

In [272]:
tst_t2 = torch.Tensor([[5, 3, 2, 5, 1, 1]]).reshape(-1, 1)
tst_t2

tensor([[5.],
        [3.],
        [2.],
        [5.],
        [1.],
        [1.]])

In [274]:
tst_t2.reshape(1,-1)[0]


tensor([5., 3., 2., 5., 1., 1.])

In [292]:
torch.tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]]).reshape(1,-1)[0]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [3]:
max([1,2,3,4,3,2,1])
np.argmax([1,2,3,4,3,2,1])

3

In [4]:
np.argmax([1,2,3,4,3,4,4,4,4,2,1])

3

In [29]:
[1,2,3,4,3,4,4,4,4,2,1][:2]

[1, 2]

In [88]:
np.random.seed(10)
sample_ids = random.sample(range(10), int(5 * 2))
sample_ids

[1, 2, 7, 5, 3, 6, 9, 4, 8, 0]

In [76]:
random.seed(10)
sample_ids = random.sample(range(10), int(5 * 2))
sample_ids

[9, 0, 6, 3, 4, 8, 1, 7, 2, 5]